In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import random
import time
import datetime
import io

In [2]:
dlg_list=[]
with io.open("data/textandembedding/allmorptok_inputtextfile.txt", encoding='euc-kr') as f:
    for a in f:
        b = a.strip().split('\t')
        dlg_list.append(b)

dlg_df=pd.DataFrame(dlg_list, columns=["dialog",'label'])
dlg_df["label"] = pd.to_numeric(dlg_df["label"])

In [3]:
type(dlg_df["label"][0])

numpy.int64

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [4]:
dialog_data_shuffled = dlg_df.sample(frac=1).reset_index(drop=True)
dialog_data_shuffled_token = dialog_data_shuffled.copy()
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
        dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_6896/678777233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tlsgh\AppData\Local\Temp/ipykernel_6896/678777233.py", line 4, in <module>
    dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\series.py", line 1056, in __setitem__
    cacher_needs_updating = self._check_is_chained_assignment_possible()
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\series.py", line 1211, in _check_is_chained_assignment_possible
    if ref is not None and ref._is_mixed_type:
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\generic.py", line 5592, in _is_mixed_type
    return self.dtypes.nunique() > 1
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-pac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tlsgh\AppData\Local\Temp/ipykernel_6896/678777233.py", line 4, in <module>
    dialog_data_shuffled_token["dialog"][idx] = tokenizer.tokenize("[CLS] " + text + " [SEP]")
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\series.py", line 1056, in __setitem__
    cacher_needs_updating = self._check_is_chained_assignment_possible()
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\series.py", line 1211, in _check_is_chained_assignment_possible
    if ref is not None and ref._is_mixed_type:
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-packages\pandas\core\generic.py", line 5592, in _is_mixed_type
    return self.dtypes.nunique() > 1
  File "C:\Users\tlsgh\anaconda3\envs\fashionhow\lib\site-pac

TypeError: object of type 'NoneType' has no len()

In [ ]:
dialog_data_shuffled_token

In [ ]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    dialog_data_shuffled_token["dialog"][idx] = dialog_data_shuffled_token["dialog"][idx][:512]

In [ ]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    textlen = len(text)
    if textlen == 512:
        dialog_data_shuffled_token["dialog"][idx][511] = "[SEP]"

In [ ]:
max10=0
for lt in dialog_data_shuffled_token["dialog"]:
    if max10 < len(lt):
        max10=len(lt)
print(max10)

In [ ]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    textlen = len(text)
    if textlen == 512:
        text[-1]

In [ ]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    dialog_data_shuffled_token["dialog"][idx] = tokenizer.convert_tokens_to_ids(text)

In [12]:
dialog_data_shuffled_token["dialog"][0]


[101,
 9521,
 118741,
 9952,
 9435,
 48549,
 9812,
 48446,
 9361,
 9645,
 48345,
 9294,
 119137,
 9633,
 9087,
 12638,
 9113,
 85836,
 118671,
 48549,
 9638,
 35465,
 9565,
 49543,
 10013,
 11287,
 9137,
 8843,
 52560,
 8898,
 19653,
 8887,
 55670,
 29805,
 9299,
 118744,
 10739,
 9233,
 8847,
 8887,
 103155,
 9638,
 9559,
 48549,
 9299,
 118744,
 9638,
 8847,
 9137,
 9645,
 9633,
 9664,
 58931,
 9954,
 9588,
 9633,
 9765,
 38631,
 9960,
 9689,
 9435,
 48549,
 9011,
 9655,
 14040,
 9248,
 8932,
 11903,
 26737,
 9689,
 9435,
 48549,
 9542,
 8888,
 9487,
 70122,
 17138,
 9638,
 9647,
 9043,
 9519,
 10739,
 108366,
 9633,
 9996,
 24974,
 9960,
 9565,
 49543,
 9559,
 9645,
 8932,
 9685,
 9632,
 9812,
 48446,
 9645,
 48345,
 9457,
 15184,
 9332,
 9128,
 9911,
 60479,
 9294,
 118905,
 9619,
 9202,
 9580,
 9043,
 9717,
 9632,
 8934,
 9638,
 9332,
 9128,
 9968,
 12692,
 9744,
 12692,
 9318,
 12508,
 9575,
 9479,
 118856,
 15184,
 9689,
 118920,
 25503,
 96567,
 9664,
 9640,
 9812,
 119372,
 91

In [13]:
for idx, text in enumerate(dialog_data_shuffled_token["dialog"]):
    text_len=len(text)
    dialog_data_shuffled_token["dialog"][idx] = text + [0] * (512-text_len)

C:\Users\tlsgh\AppData\Local\Temp/ipykernel_22292/2015811903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dialog_data_shuffled_token["dialog"][idx] = text + [0] * (512-text_len)


In [14]:
len(dialog_data_shuffled_token["dialog"][0])

512

In [15]:
train = dialog_data_shuffled_token[:47170]
test = dialog_data_shuffled_token[47170:]

In [16]:
train

,dialog,label
0,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
1,"[101, 9546, 12424, 9580, 9435, 48549, 8888, 11...",0
2,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
3,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
4,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",0
...,...,...
47165,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",0
47166,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1
47167,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",0
47168,"[101, 9521, 118741, 9952, 9435, 48549, 9812, 4...",1


In [17]:
max = 0
for text1 in train["dialog"]:
    if max < len(text1):
        max = len(text1)
print(max)
max1 = 0
for text2 in test["dialog"]:
    if max1 < len(text2):
        max1 = len(text2)
print(max1)



512
512


In [18]:
input_ids = list(dialog_data_shuffled_token["dialog"])
labels = list(dialog_data_shuffled_token["label"])

In [19]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [20]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,labels,random_state=2000,test_size=0.2)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,input_ids,random_state=2000,test_size=0.2)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [21]:
batch_size = 4

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 SUPER


In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [24]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



In [25]:
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [26]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  11,793.    Elapsed: 0:01:57.
  Batch 1,000  of  11,793.    Elapsed: 0:03:54.
  Batch 1,500  of  11,793.    Elapsed: 0:05:52.
  Batch 2,000  of  11,793.    Elapsed: 0:07:50.
  Batch 2,500  of  11,793.    Elapsed: 0:09:53.
  Batch 3,000  of  11,793.    Elapsed: 0:12:00.
  Batch 3,500  of  11,793.    Elapsed: 0:14:03.
  Batch 4,000  of  11,793.    Elapsed: 0:16:05.
  Batch 4,500  of  11,793.    Elapsed: 0:18:10.
  Batch 5,000  of  11,793.    Elapsed: 0:20:12.
  Batch 5,500  of  11,793.    Elapsed: 0:22:14.
  Batch 6,000  of  11,793.    Elapsed: 0:24:16.
  Batch 6,500  of  11,793.    Elapsed: 0:26:17.
  Batch 7,000  of  11,793.    Elapsed: 0:28:19.
  Batch 7,500  of  11,793.    Elapsed: 0:30:20.
  Batch 8,000  of  11,793.    Elapsed: 0:32:18.
  Batch 8,500  of  11,793.    Elapsed: 0:34:16.
  Batch 9,000  of  11,793.    Elapsed: 0:36:16.
  Batch 9,500  of  11,793.    Elapsed: 0:38:16.
  Batch 10,000  of  11,793.    Elapsed: 0:40:

In [30]:
torch.save(model,"./data/model/bert-4.pt")

In [99]:
torch.save(model.state_dict(), "./data/model/bert_state-4.pt")

In [44]:
import torch,gc
gc.collect()
torch.cuda.empty_cache()

In [31]:
input_ids[0]

[101,
 9521,
 118741,
 9952,
 9435,
 48549,
 9812,
 48446,
 9361,
 9645,
 48345,
 9294,
 119137,
 9633,
 9087,
 12638,
 9113,
 85836,
 118671,
 48549,
 9638,
 35465,
 9565,
 49543,
 10013,
 11287,
 9137,
 8843,
 52560,
 8898,
 19653,
 8887,
 55670,
 29805,
 9299,
 118744,
 10739,
 9233,
 8847,
 8887,
 103155,
 9638,
 9559,
 48549,
 9299,
 118744,
 9638,
 8847,
 9137,
 9645,
 9633,
 9664,
 58931,
 9954,
 9588,
 9633,
 9765,
 38631,
 9960,
 9689,
 9435,
 48549,
 9011,
 9655,
 14040,
 9248,
 8932,
 11903,
 26737,
 9689,
 9435,
 48549,
 9542,
 8888,
 9487,
 70122,
 17138,
 9638,
 9647,
 9043,
 9519,
 10739,
 108366,
 9633,
 9996,
 24974,
 9960,
 9565,
 49543,
 9559,
 9645,
 8932,
 9685,
 9632,
 9812,
 48446,
 9645,
 48345,
 9457,
 15184,
 9332,
 9128,
 9911,
 60479,
 9294,
 118905,
 9619,
 9202,
 9580,
 9043,
 9717,
 9632,
 8934,
 9638,
 9332,
 9128,
 9968,
 12692,
 9744,
 12692,
 9318,
 12508,
 9575,
 9479,
 118856,
 15184,
 9689,
 118920,
 25503,
 96567,
 9664,
 9640,
 9812,
 119372,
 91

In [60]:
outputs = model(torch.tensor(input_ids[0]).to(device), token_type_ids=None,
                      attention_mask=torch.tesnor(attention_masks[0]).to(device))

AttributeError: module 'torch' has no attribute 'tesnor'

In [80]:
len(input_ids[0])
len(attention_masks[0])


512

In [3]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [4]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
device = torch.device("cuda")
model = torch.load("data/model/bert-4.pt")
model.eval()
logits = test_sentences(["가을 축제 에 입 고 갈 스타일 로 코디 해 주 세요 치마 바지 원피스 중 어떤 옷 이 포함 된 코디 를 추천 해드릴까요 ? 원피스 나 치마 로 추천 해 주 세요 가을 에 입 기 좋 은 적당 한 두께 감 의 아우 터 와 함께 추천 해드릴까요 ? 네 상의 는 무늬 없 는 티 로 추천 해 드리 는 것 은 어떠신가요 ? 좋 습니다 얕 은 라운드 네크라인 슬림 한 일자 소매통 일자 형 암홀 손등 까지 내려오 는 소매 기장 허리선 까지 오 는 기장 전체 적 으로 슬림 한 라인 네크라인 재 원단 처리 소매 끝 과 밑단 삼봉 봉제 처리 앞 , 뒤 길 이 차이 없 는 일자 형 밑단 적당히 몸 에 붙 는 슬림 한 스타일 심플 하고 베이직 한 아이템 면 100 % 싱글 저 지 부드럽 고 편안 한 느낌 의 신축성 좋 음 두께 감 얇 음 베이지 다크 그레 이쉬 톤 의 내추럴 하 고 온화 한 느낌 편안 한 캐주얼 한 슬림 한 심플 한"])

In [6]:
type(logits[0][1])

numpy.float32

In [8]:
logits[0][1] + 0.0

-0.2473096251487732